### 0. Import

In [7]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. mon_features.pkl

In [9]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = '/content/drive/MyDrive/기학/모델 학습/data/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER2 = [
    'total_transmission_time', 'std_inter_packet_time',
    'num_outgoing_packets', 'avg_incoming_burst_size',
    'cumul_packets_30pct', 'cumul_packets_10pct',
    'incoming_order_skew', 'outgoing_order_skew',
    'cumul_max', 'bigram_OO', 'avg_outgoing_order_first_30',
    'num_incoming_first_30', 'incoming_packet_ratio'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER2]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  num_outgoing_packets  \
0                        10.14               0.041168                 121.0   
1                        10.16               0.163930                  80.0   
2                        11.11               0.066661                 118.0   
3                        13.36               0.047809                 122.0   
4                        10.64               0.038760                 115.0   
...                        ...                    ...                   ...   
18995                    43.91               0.143962                 619.0   
18996                    15.60               0.019465                 552.0   
18997                    14.93               0.016411                 579.0   
18998                    19.91               0.033281                 690.0   
18999                    13.76               0.011074                 757.0   

       avg_incoming_burst_size  cumul_packets_30pct

In [10]:
# 로그 변환
log_cols = []

for col in X.columns:
    # 숫자형이고 & 음수가 하나도 없는 경우만 로그 변환 리스트에 추가
    if pd.api.types.is_numeric_dtype(X[col]) and (X[col] < 0).sum() == 0:
        log_cols.append(col)

# 리스트에 있는 컬럼들 로그 변환
if log_cols:
    X[log_cols] = np.log1p(X[log_cols])

/tmp/ipython-input-1747048184.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[log_cols] = np.log1p(X[log_cols])


### 2. model [K-NN]

#### default 학습

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Scaling 추가
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [14]:
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)

In [15]:
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
report = classification_report(y_test, y_pred)

print("=== K-NN 기본 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 기본 성능 ===
Accuracy: 0.7086842105263158
ROC-AUC: 0.9305763577827548

Classification Report:
               precision    recall  f1-score   support

           0       0.46      0.55      0.50        40
           1       0.79      0.78      0.78        40
           2       0.65      0.82      0.73        40
           3       0.79      0.85      0.82        40
           4       0.63      0.82      0.72        40
           5       0.61      0.70      0.65        40
           6       0.86      0.90      0.88        40
           7       0.72      0.65      0.68        40
           8       0.52      0.60      0.56        40
           9       0.39      0.60      0.47        40
          10       0.73      0.80      0.76        40
          11       0.76      0.65      0.70        40
          12       0.85      0.82      0.84        40
          13       0.47      0.38      0.42        40
          14       0.47      0.65      0.55        40
          15       0.63      0.72 

#### 하이퍼 파라미터 튜닝 후 학습(GridSearchCV 이용)

In [16]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan', 'euclidean']
}

grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_estimator_)

grid_pred = grid.predict(X_test)
grid_pred_proba = grid.predict_proba(X_test)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')


In [17]:
acc = accuracy_score(y_test, grid_pred)
roc_auc = roc_auc_score(y_test, grid_pred_proba, multi_class="ovr")
report = classification_report(y_test, grid_pred)

print("=== K-NN 튜닝 후 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 튜닝 후 성능 ===
Accuracy: 0.79
ROC-AUC: 0.9318069008958567

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.65      0.69        40
           1       0.79      0.85      0.82        40
           2       0.84      0.90      0.87        40
           3       0.85      0.88      0.86        40
           4       0.82      0.80      0.81        40
           5       0.85      0.85      0.85        40
           6       0.92      0.90      0.91        40
           7       0.82      0.70      0.76        40
           8       0.76      0.65      0.70        40
           9       0.77      0.75      0.76        40
          10       0.82      0.82      0.82        40
          11       0.94      0.80      0.86        40
          12       0.89      0.85      0.87        40
          13       0.68      0.53      0.59        40
          14       0.64      0.72      0.68        40
          15       0.81      0.75      0.78   